In [1]:
import torch
import math
import pandas as pd
import numpy as np
from cayleypy import PermutationGroups, CayleyGraph
import pandas as pd
from cayleypy.models.models import ModelConfig, MlpModel

c:\Users\zamko\Documents\CaylePy\cayleypy\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def get_valid_k(n):
    # мне нужно вернуть лист из потенциальных валидных k. k является валидным если наибольший общий делитель n и k равен 1
    valid_k = []
    for k in range(1, n):
        if math.gcd(n, k) == 1:
            valid_k.append(k)
    return valid_k


def get_n_long_permutations(n):
    """Generates long permutations for n elements."""
    list_long_permutations = []
    k = int(n/2)
    if n == 2*k + 1:  # For odd "n"
        for element_index in range(n):
            p = np.arange(n)
            for i in range(k):
                p[element_index-i], p[(element_index+1+i)%n] = p[(element_index+1+i)%n], p[element_index-i]
            list_long_permutations.append(p)
    else:
        # First generate permutations corresponding to symmetries NOT passing by edges of n-gon
        for element_index in range(int(n/2)): 
            p = np.arange(n)
            for i in range(k):
                p[element_index-i], p[(element_index+1+i)%n] = p[(element_index+1+i)%n], p[element_index-i]
            list_long_permutations.append(p)
        # Second generate those which correspond to diagonal passing through the nodes
        for element_index in range(int(n/2)): 
            p = np.arange(n)
            for i in range(1, k+1):
                p[element_index-i], p[(element_index+i)%n] = p[(element_index+i)%n], p[element_index-i]
            list_long_permutations.append(p)    
            
    return list_long_permutations

def train_epoch(model, X, y, batch_size, criterion, optimizer, epoch):
    y_train = y.float()
    indices = torch.randperm(X.shape[0], dtype=torch.int64, device='cpu')  # Generate indices on CPU
    X_train = X[indices]
    y_train = y_train[indices]
    
    model.train()
    # Neural network train by batches (not to crash RAM)
    n_states_all =  X_train.shape[0]
    cc = 0; train_loss =    0.0
    for i_start_batch  in range(0,n_states_all,batch_size ):
        i_end_batch = min( [i_start_batch + batch_size,  n_states_all ] )
        
        # Forward
        outputs = model(X_train[i_start_batch:i_end_batch])
        loss = criterion(outputs.squeeze(), y_train[i_start_batch:i_end_batch])

        # Backward and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.item(); 
        cc+=1
    train_loss /= cc
    
    # print(f"Epoch {epoch:2d} | "
    #       f"Train: {train_loss:.4f} | ")
    return train_loss

def initialize_states(list_generators, device):
    n = len(list_generators[0])
    p = np.arange(n)
    # Swap (0,2)
    p[0], p[1] = p[1], p[0]
    i = 2
    while i < n-i+1:
        #print(i, n-i+1)
        p[i], p[n-i+1] = p[n-i+1], p[i]
        i += 1
    permutation_longest = torch.tensor(p, dtype=torch.int64, device=device)
    state_start = permutation_longest

    state_destination = torch.arange(len(list_generators[0]), device=device, dtype=torch.int64)
    
    return state_start, state_destination

In [3]:
# ======================= ОПРЕДЕЛЕНИЕ ПАРАМЕТРОВ =======================

# Параметры графа
n = 15
valid_k = get_valid_k(n)
k = valid_k[0]

CFG = dict()

# Параметры сети
CFG["model_type"] = "MLP"
CFG["input_size"] = n
CFG["num_classes_for_one_hot"] = n
CFG["layers_sizes"] = [4096]
CFG["weights_kaggle_id"] = None
CFG["weights_path"] = None
CFG["learning_rate"] = 0.001
# Параметры сети

# Параметры обучения
CFG["batch_size"] = 1024
CFG["num_epochs"] = 100

# Параметры beam search
CFG["num_start_states"] = 5
CFG["beam_size_range"] = [10**2]
CFG["max_iterations"] = 1024
CFG["return_path"] = True
CFG["bfs_result_for_mitm"] = None

# random_walks parameters
CFG["k"] = k
CFG["random_walks_width"] = 1024
CFG["random_walks_length"] = n * (n - 1) // 2

CFG_model = ModelConfig.from_dict(CFG)

In [4]:
from tqdm import tqdm
# ======================= ОСНОВНАЯ ЛОГИКА =======================
df_result = pd.DataFrame(columns=["n", "k", "start_state", "beam_size", "path_found", "path_length", "path"])

# 1. Создание графа
graph = CayleyGraph(PermutationGroups.lrx(CFG["input_size"], CFG["k"]), bit_encoding_width = None)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 2. Создание модели
model = MlpModel(CFG_model).to(device)

# 4. Определение функции потерь и оптимизатора
loss_fn = torch.nn.MSELoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=CFG["learning_rate"], weight_decay= 1e-5)

# 5. Цикл обучения
pbar = tqdm(range(CFG["num_epochs"]), desc="Training MLP with random walks")
for epoch in pbar:
    X_tr, y_tr = graph.random_walks(width=CFG["random_walks_width"], length=CFG["random_walks_length"], mode="bfs")
    y_tr = y_tr.float()
    loss = train_epoch(model, X_tr, y_tr, CFG["batch_size"], loss_fn, optimizer, epoch)
    if (epoch + 1) % 10 == 0:
        # обновляем только postfix, не ломая самую прогресс-строку
        pbar.set_postfix(loss=f"{loss:.4f}")

# 6. Запуск beam search и сравнение методов
print("Starting beam search with hamming distance predictor...")
nn_success_rates = {}
list_long_permutations = get_n_long_permutations(n)
list_long_permutations = [torch.tensor(start_state, dtype=torch.int64) for start_state in list_long_permutations]
for start_state in list_long_permutations:
    nn_success_rates[start_state] = graph.beam_search(start_state=start_state, predictor=model, beam_width=CFG["beam_size_range"][-1], max_iterations=CFG["max_iterations"], return_path=CFG["return_path"], bfs_result_for_mitm=CFG["bfs_result_for_mitm"])

Training MLP with random walks: 100%|██████████| 100/100 [00:40<00:00,  2.46it/s, loss=207.4911]


Starting beam search with hamming distance predictor...


In [5]:
nn_success_rates

{tensor([ 1,  0, 14, 13, 12, 11, 10,  9,  8,  7,  6,  5,  4,  3,  2]): BeamSearchResult(path_found=False),
 tensor([ 3,  2,  1,  0, 14, 13, 12, 11, 10,  9,  8,  7,  6,  5,  4]): BeamSearchResult(path_found=False),
 tensor([ 5,  4,  3,  2,  1,  0, 14, 13, 12, 11, 10,  9,  8,  7,  6]): BeamSearchResult(path_found=False),
 tensor([ 7,  6,  5,  4,  3,  2,  1,  0, 14, 13, 12, 11, 10,  9,  8]): BeamSearchResult(path_found=False),
 tensor([ 9,  8,  7,  6,  5,  4,  3,  2,  1,  0, 14, 13, 12, 11, 10]): BeamSearchResult(path_found=False),
 tensor([11, 10,  9,  8,  7,  6,  5,  4,  3,  2,  1,  0, 14, 13, 12]): BeamSearchResult(path_found=False),
 tensor([13, 12, 11, 10,  9,  8,  7,  6,  5,  4,  3,  2,  1,  0, 14]): BeamSearchResult(path_length=125, path=X.R.X.L.X.R.R.R.R.R.X.L.X.L.X.R.X.R.X.L.X.R.R.R.X.L.X.R.X.L.L.X.L.X.R.R.X.L.X.L.L.X.R.X.L.L.L.L.X.R.X.R.X.R.X.R.X.R.X.R.X.R.X.R.R.R.X.R.X.R.X.R.X.R.X.L.X.L.X.L.X.L.X.L.X.R.X.R.X.R.X.R.X.R.X.R.X.L.X.L.X.L.X.L.X.L.X.R.X.R.X.R.X.L.X.L.X.R.X.L.L.L.L.L.